<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Doda-now-kolumn---ilo-nagrd-dla-danego-aktora/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

In [ ]:
#!git checkout

In [ ]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

In [ ]:
import pandas as pd
from lib.pyspark_init import create_spark_context, load_data
from lib.const import JOINED_DATA

In [ ]:
spark = create_spark_context()
spark

In [ ]:
!chmod +x download_data.sh
!./download_data.sh

In [ ]:
%%time
data = load_data(spark)
data.show(3)

In [ ]:
%%time
data.write.parquet(JOINED_DATA)

# Dodanie kolumn z policzonymi zdobytymi nagrodami

In [ ]:
oscars_nominations = data.select("*", explode("winner_oscars").alias("exploded"))\
    .groupBy("nconst", "winner_oscars")\
    .agg(count("exploded").alias("no_nominations_oscars"))\
    .select(["nconst", "no_nominations_oscars"])

oscars_win = data.select("*", explode("winner_oscars").alias("exploded"))\
    .where(col("exploded") == True)\
    .groupBy("nconst", "winner_oscars")\
    .agg(count("exploded").alias("no_oscars"))\
    .select(["nconst", "no_oscars"])

data = data.join(oscars_nominations, on="nconst", how="left")
data = data.join(oscars_win, on="nconst", how="left")

In [ ]:
globes_nominations = data.select("*", explode("win_globes").alias("exploded"))\
    .groupBy("nconst", "win_globes")\
    .agg(count("exploded").alias("no_nominations_globes"))\
    .select(["nconst", "no_nominations_globes"])

globes_win = data.select("*", explode("win_globes").alias("exploded"))\
    .where(col("exploded") == True)\
    .groupBy("nconst", "win_globes")\
    .agg(count("exploded").alias("no_globes"))\
    .select(["nconst", "no_globes"])

data = data.join(globes_nominations, on="nconst", how="left")    
data = data.join(globes_win, on="nconst", how="left")

In [ ]:
emmy_nominations = data.select("*", explode("win_emmy").alias("exploded"))\
    .groupBy("nconst", "win_emmy")\
    .agg(count("exploded").alias("no_nominations_emmy"))\
    .select(["nconst", "no_nominations_emmy"])

emmy_win = data.select("*", explode("win_emmy").alias("exploded"))\
    .where(col("exploded") == True)\
    .groupBy("nconst", "win_emmy")\
    .agg(count("exploded").alias("no_emmy"))\
    .select(["nconst", "no_emmy"])

data = data.join(emmy_nominations, on="nconst", how="left")    
data = data.join(emmy_win, on="nconst", how="left")

In [ ]:
data.show(10)